## Set the enviroments.

#### Connect to slime.

In [8]:
(length (core:bformat nil "%16.8e" 1234.5676))

16

In [1]:
(start-swank)

T

#### Add paths.

In [2]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [3]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [4]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [5]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [6]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [19]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [20]:
(build-hydrogens *chemagg*)

NIL

In [21]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [22]:
(nglv::show-aggregate *chemagg*)

An error occurred of type: UNDEFINED-FUNCTION: 
  #<UNDEFINED-FUNCTION>


NIL

#### Gather atoms to the center.

In [23]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C_0    S


NIL

#### Set the bounding box.

#### Assign atom types and change one of them for alanine atom type definition.

In [24]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C_0)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [25]:

(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-name a) :C_0)
                                     (format t "type ~a name ~a " (chem:get-type a) (chem:get-name a))))
                             *chemagg*)

type CX name C_0 

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [26]:
(cando:jostle *chemagg*)

NIL

In [27]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          13290.040
 minSDnP       0     2     -7.97   0.0        2338253.847           6446.298
DONE absolute force test:
forceRmsMag(4751.408305).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     3      -inf   0.0              0.000           4751.408
 minCGnP       0     4     -7.83  38.6        1633207.994           3787.039
 minCGnP       0     5     -6.97  56.8        1206503.675           5209.188
 minCGnP       0     6     -8.75  45.2        1122755.696           2950.402
 minCGnP       0     7     -7.64  42.6         980612.237           2086.800
 minCGnP       0     8     -7.05  66.5         853925.063           3643.118
 minCGnP       0     9     -7.61  64.5         671374.143           3957.363
 minCGnP       0    10     -8.07  79.5   

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [28]:
(nglv::show-aggregate *chemagg*)

An error occurred of type: UNDEFINED-FUNCTION: 
  #<UNDEFINED-FUNCTION>


NIL

In [29]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [30]:
(ext:vfork-execvp (list (namestring (translate-logical-pathname #P"amber:bin;sqm"))
                             "-O"
                             "-i" "/tmp/sqm-input.txt"
                             "-o" "/tmp/sqm-output.out"))

0

In [36]:
(charges:calculate-am1-bcc-charges *chemagg*)

SIMPLE-ERROR: 
Mismatch atom name C_0 element C element 2


NIL

In [34]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/alanine.mol2


NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [35]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C_0)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [37]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" :assign-types nil)

T

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [92]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [93]:
(print ~tip3pfbox)


#<AGGREGATE NIL> 

#<AGGREGATE NIL>

In [94]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [95]:
(nglv::show-aggregate *solvent*)

An error occurred of type: UNDEFINED-FUNCTION: 
  #<UNDEFINED-FUNCTION>


NIL

#### Assign the atom types of the force field to the water box.

In [96]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [97]:
(leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0))

Total bounding box for atom centers:  29.353789041228794d0 26.304757298876325d0 23.279665206725056d0


NIL

In [66]:
(nglv::show-aggregate *newagg*)

An error occurred of type: UNDEFINED-FUNCTION: 
  #<UNDEFINED-FUNCTION>


NIL

In [98]:
(show *chemagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [99]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [100]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil)

T

In [ ]:
(load-mol2 "alanine-leap.mol2")

#### Calculate each energy parameters.

In [31]:
(chem:get-energy (chem:get-stretch-component *e*))

1720.1189821006878d0

In [32]:
(chem:get-energy (chem:get-angle-component *e*))

0.30171081698364277d0

In [33]:
(chem:get-energy (chem:get-dihedral-component *e*))

5.456751705606037d0

In [34]:
(chem:get-energy (chem:get-nonbond-component *e*))

658.8024508583202d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL

In [88]:
(:= w (show *chemagg*)) w

A Jupyter Widget

In [93]:
(nglv::picked w)

(("atom" ("index" . 20) ("residueIndex" . 2) ("resname" . "NIL_3")
  ("x" . 216.329) ("y" . 322.549) ("z" . -0.62) ("element" . "H")
  ("chainname" . "A") ("resno" . 3) ("serial" . 21) ("vdw" . 1.1)
  ("covalent" . 0.31) ("hetero" . 1) ("bfactor" . 0) ("altloc" . "")
  ("atomname" . "HH33")))

In [94]:
(defun distance (p1 p2) (geom:vlength (geom:v- p1 p2)))

DISTANCE

In [95]:
(:= *a* (let* ((picked (nglv::picked w))
       (atom (cdr (assoc "atom" picked :test #'string=)))
       (x (cdr (assoc "x" atom :test #'string=)))
       (y (cdr (assoc "y" atom :test #'string=)))
       (z (cdr (assoc "z" atom :test #'string=)))
       (pos (geom:vec x y z))
        resulta)
     (chem:map-atoms nil (lambda (a) 
                                 (when (< (distance pos (chem:get-position a)) 0.01) 
                                     (setf resulta a))) *chemagg*)
              resulta))

*A*

In [96]:
*a*

#<ATOM :HH33/:H>

In [97]:
(:= *p* (core:make-cxx-object 'chem:chem-info))

*P*

In [98]:
(chem:compile-smarts *p* "CC")

T

In [99]:
(chem:matches *p* *a*)

NIL

In [47]:
(:= *ala-leap* (load-mol2 "alanine_leap.mol2"))

*ALA-LEAP*

In [48]:
(cando::show-aggregate *ala-leap*)

A Jupyter Widget

In [49]:
(chem:map-atoms 'nil (lambda (a) (format t "type ~a name ~a~% " (chem:get-type a) (chem:get-name a)))
                
                             *ala-leap*)

type HC name H1
 type CT name CH3
 type HC name H2
 type HC name H3
 type C name C
 type O name O
 type N name N
 type H name H
 type CX name CA
 type H1 name HA
 type CT name CB
 type HC name HB1
 type HC name HB2
 type HC name HB3
 type C name C
 type O name O
 type N name N
 type H name H
 type CT name CH3
 type H1 name HH31
 type H1 name HH32
 type H1 name HH33
 

NIL

In [50]:
(leap.topology:save-amber-parm-format *ala-leap* "alanine_cando.top" "alanine_cando.crd" :assign-types nil)

T

In [42]:
(:= *ala-pdb* (load-pdb "ace-ala-nme.pdb"))

popped residue: #<PDB-RESIDUE ACE>
popped residue: #<PDB-RESIDUE ALA>
popped residue: #<PDB-RESIDUE NME>
Built 3 missing hydrogens


SIMPLE-WARNING: 
Loaded atom (1 H1 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it
SIMPLE-WARNING: 
Loaded atom (3 H2 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it
SIMPLE-WARNING: 
Loaded atom (4 H3 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it


*ALA-PDB*

In [25]:
(cando::show-aggregate *ala-pdb*)

A Jupyter Widget

In [26]:
(chem:map-atoms 'nil (lambda (a) (format t "type ~a name ~a~% " (chem:get-type a) (chem:get-name a)))
                
                             *ala-leap*)

type HC name H1
 type CT name CH3
 type HC name H2
 type HC name H3
 type C name C
 type O name O
 type N name N
 type H name H
 type CX name CA
 type H1 name HA
 type CT name CB
 type HC name HB1
 type HC name HB2
 type HC name HB3
 type C name C
 type O name O
 type N name N
 type H name H
 type CT name CH3
 type H1 name HH31
 type H1 name HH32
 type H1 name HH33
 

NIL

In [27]:
(leap.topology:save-amber-parm-format *ala-pdb* "alanine_pdb.top" "alanine_pdb.crd" :assign-types nil)

T